In [1]:
using DataFrames, CSV, Plots
ENV["ROWS"]=120

120

In [2]:
fuel_data = CSV.read("Fuels_data.csv", DataFrame)

Row,Fuel,Cost_per_MMBtu,CO2_content_tons_per_MMBtu,fuel_indices
,String31,Float64,Float64,Int64
1,None,0.0,0.0,1
2,ercot_coal,1.91,0.09552,2
3,ercot_naturalgas,3.62,0.05306,3
4,ercot_uranium,0.71,0.0,4
5,ercot_naturalgas_ccs90,4.1,0.00531,5
6,ercot_naturalgas_ccs100,4.15,0.0,6


In [7]:
generator_data = CSV.read("Generators_data.csv", DataFrame)
generator_technology = unique(generator_data[!,:Resource])
# We remove interconnectors first
generator_technology = generator_technology[(.!occursin.("losangeles", generator_technology)) .& (.!occursin.("ltrg1",generator_technology)), :];
# We remove average capacity factor gas
generator_technology = generator_technology[.!occursin.("avgcf", generator_technology), :];
# We remove repeated natural gas
generator_technology = generator_technology[.!occursin.("naturalgas", generator_technology), :];
# We remove steam turbine as well
generator_technology = generator_technology[.!occursin.("steam_turbine", generator_technology), :];

# From those technologies, we check the different types of generators available to get the "average"
for tech in generator_technology
    gen = generator_data[generator_data[!,:Resource] .== tech, :]
    println(gen)
end


3×74 DataFrame
 Row │ region   Resource              cluster  R_ID   zone   voltage_level  CapRes   THERM  DISP   NDISP  STOR   DR     HEAT     NACC     TRADER   HYDRO  RPS    CES    Commit  Min_Share  Max_Share  Existing_Cap_MWh  Existing_Cap_MW  num_units  unmodified_existing_cap_mw  New_Build  Cap_size  Min_Cap_MW  Max_Cap_MW  Min_Share_percent  Max_Share_percent  capex    Inv_cost_per_MWyr  Fixed_OM_cost_per_MWyr  capex_mwh  Inv_cost_per_MWhyr  Fixed_OM_cost_per_MWhyr  Var_OM_cost_per_MWh  Var_OM_cost_per_MWh_in  Externality_cost_MWh  Start_cost_per_MW  Start_fuel_MMBTU_per_MW  Heat_rate_MMBTU_per_MWh  heat_rate_mmbtu_mwh_iqr  heat_rate_mmbtu_mwh_std  Fuel      Min_power  Self_disch  Eff_up   Eff_down  Ratio_power_to_energy  Min_Duration  Max_Duration  Max_DSM_delay  Ramp_Up_percentage  Ramp_Dn_percentage  Up_time  Down_time  NACC_Eff  NACC_Peak_to_Base  Reg_Up   Reg_Dn   Rsv_Up   Rsv_Dn   Reg_Cost  Rsv_Cost  spur_miles  spur_line_capex  interconnect_annuity  Power_uprate_cost_per_

1×74 DataFrame
 Row │ region   Resource                    cluster  R_ID   zone   voltage_level  CapRes   THERM  DISP   NDISP  STOR   DR     HEAT     NACC     TRADER   HYDRO  RPS    CES    Commit  Min_Share  Max_Share  Existing_Cap_MWh  Existing_Cap_MW  num_units  unmodified_existing_cap_mw  New_Build  Cap_size  Min_Cap_MW  Max_Cap_MW  Min_Share_percent  Max_Share_percent  capex    Inv_cost_per_MWyr  Fixed_OM_cost_per_MWyr  capex_mwh  Inv_cost_per_MWhyr  Fixed_OM_cost_per_MWhyr  Var_OM_cost_per_MWh  Var_OM_cost_per_MWh_in  Externality_cost_MWh  Start_cost_per_MW  Start_fuel_MMBTU_per_MW  Heat_rate_MMBTU_per_MWh  heat_rate_mmbtu_mwh_iqr  heat_rate_mmbtu_mwh_std  Fuel      Min_power  Self_disch  Eff_up   Eff_down  Ratio_power_to_energy  Min_Duration  Max_Duration  Max_DSM_delay  Ramp_Up_percentage  Ramp_Dn_percentage  Up_time  Down_time  NACC_Eff  NACC_Peak_to_Base  Reg_Up   Reg_Dn   Rsv_Up   Rsv_Dn   Reg_Cost  Rsv_Cost  spur_miles  spur_line_capex  interconnect_annuity  Power_uprate_cos

2×74 DataFrame
 Row │ region   Resource                          cluster  R_ID   zone   voltage_level  CapRes   THERM  DISP   NDISP  STOR   DR     HEAT     NACC     TRADER   HYDRO  RPS    CES    Commit  Min_Share  Max_Share  Existing_Cap_MWh  Existing_Cap_MW  num_units  unmodified_existing_cap_mw  New_Build  Cap_size  Min_Cap_MW  Max_Cap_MW  Min_Share_percent  Max_Share_percent  capex    Inv_cost_per_MWyr  Fixed_OM_cost_per_MWyr  capex_mwh  Inv_cost_per_MWhyr  Fixed_OM_cost_per_MWhyr  Var_OM_cost_per_MWh  Var_OM_cost_per_MWh_in  Externality_cost_MWh  Start_cost_per_MW  Start_fuel_MMBTU_per_MW  Heat_rate_MMBTU_per_MWh  heat_rate_mmbtu_mwh_iqr  heat_rate_mmbtu_mwh_std  Fuel              Min_power  Self_disch  Eff_up   Eff_down  Ratio_power_to_energy  Min_Duration  Max_Duration  Max_DSM_delay  Ramp_Up_percentage  Ramp_Dn_percentage  Up_time  Down_time  NACC_Eff  NACC_Peak_to_Base  Reg_Up   Reg_Dn   Rsv_Up   Rsv_Dn   Reg_Cost  Rsv_Cost  spur_miles  spur_line_capex  interconnect_annuity  Po

1×74 DataFrame
 Row │ region   Resource  cluster  R_ID   zone   voltage_level  CapRes   THERM  DISP   NDISP  STOR   DR     HEAT     NACC     TRADER   HYDRO  RPS    CES    Commit  Min_Share  Max_Share  Existing_Cap_MWh  Existing_Cap_MW  num_units  unmodified_existing_cap_mw  New_Build  Cap_size  Min_Cap_MW  Max_Cap_MW  Min_Share_percent  Max_Share_percent  capex    Inv_cost_per_MWyr  Fixed_OM_cost_per_MWyr  capex_mwh  Inv_cost_per_MWhyr  Fixed_OM_cost_per_MWhyr  Var_OM_cost_per_MWh  Var_OM_cost_per_MWh_in  Externality_cost_MWh  Start_cost_per_MW  Start_fuel_MMBTU_per_MW  Heat_rate_MMBTU_per_MWh  heat_rate_mmbtu_mwh_iqr  heat_rate_mmbtu_mwh_std  Fuel           Min_power  Self_disch  Eff_up   Eff_down  Ratio_power_to_energy  Min_Duration  Max_Duration  Max_DSM_delay  Ramp_Up_percentage  Ramp_Dn_percentage  Up_time  Down_time  NACC_Eff  NACC_Peak_to_Base  Reg_Up   Reg_Dn   Rsv_Up   Rsv_Dn   Reg_Cost  Rsv_Cost  spur_miles  spur_line_capex  interconnect_annuity  Power_uprate_cost_per_MWyr  M

2×74 DataFrame
 Row │ region   Resource            cluster  R_ID   zone   voltage_level  CapRes   THERM  DISP   NDISP  STOR   DR     HEAT     NACC     TRADER   HYDRO  RPS    CES    Commit  Min_Share  Max_Share  Existing_Cap_MWh  Existing_Cap_MW  num_units  unmodified_existing_cap_mw  New_Build  Cap_size  Min_Cap_MW  Max_Cap_MW  Min_Share_percent  Max_Share_percent  capex    Inv_cost_per_MWyr  Fixed_OM_cost_per_MWyr  capex_mwh  Inv_cost_per_MWhyr  Fixed_OM_cost_per_MWhyr  Var_OM_cost_per_MWh  Var_OM_cost_per_MWh_in  Externality_cost_MWh  Start_cost_per_MW  Start_fuel_MMBTU_per_MW  Heat_rate_MMBTU_per_MWh  heat_rate_mmbtu_mwh_iqr  heat_rate_mmbtu_mwh_std  Fuel      Min_power  Self_disch  Eff_up   Eff_down  Ratio_power_to_energy  Min_Duration  Max_Duration  Max_DSM_delay  Ramp_Up_percentage  Ramp_Dn_percentage  Up_time  Down_time  NACC_Eff  NACC_Peak_to_Base  Reg_Up   Reg_Dn   Rsv_Up   Rsv_Dn   Reg_Cost  Rsv_Cost  spur_miles  spur_line_capex  interconnect_annuity  Power_uprate_cost_per_MW

In [4]:
demand_data = CSV.read("Load_data.csv", DataFrame)
demand_data = select(demand_data, :Time_index, :Load_MW_z2, :Load_MW_z3)
demand_z3 = sort(demand_data[!,:Load_MW_z3], rev=true) #for the model, we'll use the load duration curve from zone 3

8760-element Vector{Int64}:
 5632
 5618
 5590
 5589
 5574
 5565
 5545
 5540
 5538
 5524
    ⋮
 2653
 2652
 2643
 2642
 2641
 2635
 2633
 2626
 2624